<h1>Laboratorio N° 02 - Utilizando Operadores de Conjuntos de Datos

<h2>Ejercicio 1: Operador UNION

1.1. Conectar con la BD TSQL

In [2]:
USE TSQL;
GO

Commands completed successfully.

Total execution time: 00:00:00

1.2. Write a SELECT statement to return the productid and productname columns from the Production.Products table. Filter the results to include only products that have a categoryid value 4

In [3]:
SELECT productid, productname from Production.Products WHERE categoryid = 4

(10 rows affected)

Total execution time: 00:00:00.012

productid,productname
11,Product QMVUN
12,Product OSFNS
31,Product XWOXC
32,Product NUNAW
33,Product ASTMN
59,Product UKXRI
60,Product WHBYK
69,Product COAXA
71,Product MYMOI
72,Product GEEOO


1.3. Write a SELECT statement to return the productid and productname columns from the Production.Products table. Filter the results to include only products that have a total sales amount of more than $50,000. For the total sales amount, you will need to query the Sales.OrderDetails table and aggregate all order line values (qty * unitprice) for each product

In [4]:
SELECT 
    c.productid,
    c.productname,
    c.Sales_Amount
FROM
(
    SELECT 
	S.productid,P.productname, SUM(S.qty*S.unitprice) AS Sales_Amount
	FROM Sales.OrderDetails AS S
	INNER JOIN Production.Products AS P ON S.productid = P.productid
	GROUP BY S.productid,P.productname
) AS c 
WHERE c.Sales_Amount > 50000

(4 rows affected)

Total execution time: 00:00:00.013

productid,productname,Sales_Amount
29,Product VJXYN,87736.4000
38,Product QDOMO,149984.2000
59,Product UKXRI,76296.0000
60,Product WHBYK,50286.0000


1.3. Write a SELECT statement to retrieve the custid and contactname columns from the Sales.Customers table. Display the top 10 customers by sales amount for January 2008 and display the top 10 customers by sales amount for February 2008. (Hint: write two SELECT statements, each  joining Sales.Customers and Sales.OrderValues, and use the appropriate set operator.)

In [5]:
SELECT custid, contactname FROM Sales.Customers 

(91 rows affected)

Total execution time: 00:00:00.014

custid,contactname
1,"Allen, Michael"
2,"Hassall, Mark"
3,"Peoples, John"
4,"Arndt, Torsten"
5,"Higginbotham, Tom"
6,"Poland, Carole"
7,"Bansal, Dushyant"
8,"Ilyina, Julia"
9,"Raghav, Amritansh"
10,"Bassols, Pilar Colome"


<h2>Ejercicio 2: Operador EXCEPT y INTERSECT

2.1. Write a SELECT statement to retrieve the custid column from the Sales.Orders table. Filter the results to include only customers who bought more than 20 different products (based on the productid column from the Sales.OrderDetails table)

2.2. Write a SELECT statement to retrieve the custid column from the Sales.Orders table. Filter the results to include only customers from the country USA and exclude all customers from the previous (task 1) result. (Hint: use the EXCEPT operator and the previous query.)

2.3. Write a SELECT statement to retrieve the custid column from the Sales.Orders table. Filter only customers who have a total sales value greater than $10,000. Calculate the sales value using the qty and unitprice columns from the Sales.OrderDetails table

<h2>Ejercicio 3: Utilizando el Operador APPLY

3.1. Write a SELECT statement to retrieve the productid and productname columns from the Production.Products table. In addition, for each product, retrieve the last two rows from the Sales.OrderDetails table based on orderid number.

3.2. Use the CROSS APPLY operator and a correlated subquery. Order the result by the column productid

3.3. Ejecutar la sentencia siguiente para crear la función fnGetTop3ProductsForCustomer.

In [1]:
DROP FUNCTION IF EXISTS dbo.fnGetTop3ProductsForCustomer; 
GO 
CREATE FUNCTION dbo.fnGetTop3ProductsForCustomer (@custid AS INT) 
RETURNS TABLE AS 
RETURN 
SELECT TOP(3) d.productid, p.productname, SUM(d.qty * d.unitprice) AS totalsalesamount 
FROM Sales.Orders AS o 
INNER JOIN Sales.OrderDetails AS d ON d.orderid = o.orderid 
INNER JOIN Production.Products AS p ON p.productid = d.productid 
WHERE custid = @custid 
GROUP BY d.productid, p.productname 
ORDER BY totalsalesamount DESC;

3.4. Write a SELECT statement to retrieve the custid and contactname columns from the Sales.Customers table. Use the CROSS APPLY operator with the dbo.fnGetTop3ProductsForCustomer function to retrieve productid, productname, and totalsalesamount columns for each customer